<a href="https://colab.research.google.com/github/Hmiru/Fashion-sketch-using-AI/blob/main/T2I_lineart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title 사전 준비
!pip install -U git+https://github.com/huggingface/diffusers.git
!pip install -U controlnet_aux==0.0.7 # for conditioning models and detectors
!pip install transformers accelerate safetensors
!pip install diffusers
!pip install xformers

  Cloning https://github.com/huggingface/diffusers.git to /tmp/pip-req-build-0zewno8_
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers.git /tmp/pip-req-build-0zewno8_
  Resolved https://github.com/huggingface/diffusers.git to commit 06ee4db3e7a5342871404ae445cf71665bc6a580
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
#@title ram이 부족할 때 클릭
import gc
import torch
torch.cuda.empty_cache()
gc.collect()

In [ ]:
#@title 입력 받을 준비
import io
from PIL import Image
import ipywidgets as widgets
from IPython.display import display, clear_output

# 사용자 입력을 위한 위젯
guidance_slider = widgets.FloatSlider(value=7.5, min=0.1, max=30, step=0.1, description='Guidance scale:')
adapter_slider = widgets.FloatSlider(value=0.5, min=0.0, max=1.0, step=0.1, description='Adapter conf:')
step_slider = widgets.IntSlider(value=20, min=1, max=200, step=1, description='Step:')
prompt_text = widgets.Text(
    value="redraw this figure using only lines to match the proportions of a normal person, high resolution, peleats, fashion sketch, pencil drawing, turtleneck, jacket, Enhance the overall quality of the drawing and add fine details, clean line, Maintain the pose and the form of the clothes",
    description='Positive Prompt:', layout=widgets.Layout(width='100%')
)
negative_prompt_text = widgets.Text(
    value="color, shadow, text,altered cloth shapes, several people, real person, vague line, real person, painting, mutated, disfigured, ceramic, dotted line",
    description='Negative Prompt:', layout=widgets.Layout(width='100%')
)
generate_button = widgets.Button(description="Generate Image")
reset_button = widgets.Button(description="Reset")
upload_button = widgets.FileUpload(description="Upload Image", accept='image/*', multiple=False)

# 위젯 레이아웃 설정
adapter_and_slider = widgets.HBox([guidance_slider, adapter_slider, step_slider])
buttons_and_upload = widgets.HBox([generate_button, reset_button, upload_button])
control_widgets = widgets.VBox([adapter_and_slider, prompt_text, negative_prompt_text, buttons_and_upload])


In [ ]:
#@title 기본 실행
# 위젯 표시 함수
def display_widgets():
    display(control_widgets)

# 초기 위젯 표시
display_widgets()

# 이미지 처리 함수
def process_image(guidance_scale, adapter_conf, num_steps, prompt, negative_prompt, image):
    image = detector(image, detect_resolution=384, image_resolution=1024, apply_filter=True)
    with torch.inference_mode():
        gen_images = pipe(
            prompt=prompt,
            negative_prompt=negative_prompt,
            image=image,
            num_inference_steps=num_steps,
            adapter_conditioning_scale=adapter_conf,
            guidance_scale=guidance_scale
        ).images[0]
    return gen_images

# 버튼 클릭 콜백 함수
def on_generate_button_click(b):
    clear_output(wait=True)
    display_widgets()
    if upload_button.value:
        # 업로드된 파일 가져오기
        uploaded_file = list(upload_button.value.values())[0]
        image = Image.open(io.BytesIO(uploaded_file['content']))
        gen_image = process_image(guidance_slider.value, adapter_slider.value, step_slider.value, prompt_text.value, negative_prompt_text.value, image)
        display(gen_image)
    else:
        print("Please upload an image before generating.")

def on_reset_button_click(b):
    global upload_button
    guidance_slider.value = 7.5
    adapter_slider.value = 0.5
    step_slider.value = 20
    prompt_text.value = "redraw this figure using only lines to match the proportions of a normal person, high resolution, peleats, fashion sketch, pencil drawing, turtleneck, jacket, Enhance the overall quality of the drawing and add fine details, clean line, Maintain the pose and the form of the clothes"
    negative_prompt_text.value = "color, shadow, text,altered cloth shapes, several people, real person, vague line, real person, painting, mutated, disfigured, ceramic, dotted line"
    # 파일 업로드 버튼 재설정
    upload_button = widgets.FileUpload(description="Upload Image", accept='image/*', multiple=False)
    # 이벤트 다시 연결
    generate_button.on_click(on_generate_button_click)
    reset_button.on_click(on_reset_button_click)
    clear_output(wait=True)
    display_widgets()

# 버튼 클릭 이벤트 연결
generate_button.on_click(on_generate_button_click)
reset_button.on_click(on_reset_button_click)


/content
